# Extract authors from PMC-OAI frontmatter `<article>` records

In [1]:
import pathlib

import pandas

from pubmedpy.xml import yield_etrees_from_zip
from pubmedpy.pmc_oai import extract_authors_from_article

In [2]:
zip_paths = sorted(pathlib.Path('data/pmc/oai/pmc_fm').glob('*.zip'))
zip_paths

[PosixPath('data/pmc/oai/pmc_fm/bioinfo.zip'),
 PosixPath('data/pmc/oai/pmc_fm/bmcbioi.zip'),
 PosixPath('data/pmc/oai/pmc_fm/ploscomp.zip')]

In [3]:
authors = list()
for zip_path in zip_paths:
    for name, article in yield_etrees_from_zip(zip_path):
        authors.extend(extract_authors_from_article(article))
author_df = pandas.DataFrame(authors)
author_df = author_df.sort_values(['pmcid', 'position'])
author_df.tail()

,pmcid,position,fore_name,last_name,corresponding,reverse_position
24041,PMC77394,2,Ferdinando Di,Cunto,0,2
24042,PMC77394,3,Paolo,Provero,1,1
24043,PMC90187,1,Jonas S,Almeida,1,2
24044,PMC90187,2,Susana,Vinga,0,1
24045,PMC99049,1,Harry J,Mangalam,1,1


In [4]:
# Total number of articles
author_df.pmcid.nunique()

21587

In [5]:
# number of corresponding authors per paper
n_corresponding = author_df.groupby("pmcid").corresponding.sum()
pmcids_without_corresponding = set(n_corresponding[n_corresponding == 0].index)

In [6]:
# Probability of author position being corresponding,
# given that there's at least one corresponding author
# and the author is not the last author
(
    author_df
    .query("pmcid not in @pmcids_without_corresponding")
    .query("reverse_position > 1")
    .groupby("position")
    .corresponding
    .mean()
    .map("{:.1%}".format)
    .head()
)

position
1    42.9%
2     7.6%
3     4.6%
4     4.6%
5     5.3%
Name: corresponding, dtype: object

In [7]:
# Probability of author reverse position being corresponding,
# given that there's at least one corresponding author
# and the author is not the first author
(
    author_df
    .query("pmcid not in @pmcids_without_corresponding")
    .query("position > 1")
    .groupby("reverse_position")
    .corresponding
    .mean()
    .map("{:.1%}".format)
    .head()
)

reverse_position
1    61.9%
2    12.4%
3     4.1%
4     3.0%
5     3.5%
Name: corresponding, dtype: object

In [8]:
# Corresponding author counts
n_corresponding.value_counts().sort_index()

0       371
1     17529
2      3267
3       314
4        62
5        19
6         7
7         2
8         2
9         6
10        2
11        1
14        2
15        1
17        1
21        1
Name: corresponding, dtype: int64

In [9]:
# Testing: show some articles without any corresponding authors
n_corresponding.reset_index().query("corresponding == 0").head()

,pmcid,corresponding
105,PMC1183510,0
106,PMC1183511,0
107,PMC1183512,0
119,PMC1185644,0
160,PMC1193992,0


In [10]:
# Testing: show some articles without >10 corresponding authors
n_corresponding.reset_index().query("corresponding >= 10")

,pmcid,corresponding
9078,PMC3463115,15
9349,PMC3509495,14
9393,PMC3519461,17
9583,PMC3546797,10
9719,PMC3570207,11
10363,PMC3694659,10
15564,PMC5001208,21
17344,PMC5647556,14


In [11]:
# Write author dataframe to a TSV
author_df.to_csv('data/pmc/authors.tsv.xz', index=False, sep='\t')